In [ ]:
%cd drive/My Drive/Estudos/titanic-train

/content/drive/My Drive/Estudos/titanic-train


### Carregar bibliotecas

In [ ]:
import pandas as pd
from datetime import datetime
import random
import sklearn
from sklearn import tree, neighbors, ensemble, discriminant_analysis, neural_network, naive_bayes, calibration, gaussian_process, semi_supervised
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn.decomposition import PCA, NMF, FastICA
import joblib
from sklearn.pipeline import Pipeline
import pytz

import warnings
warnings.filterwarnings('ignore')

### Carregar os dados

In [ ]:
versao_dados = '3_4'
dados = pd.read_csv('dados/train_tratado-'+versao_dados+'.csv', sep=';')
dados.head()

,Survived,Fare,Pclass_3,Sex_male,Name_Miss,Name_Mr,Name_Mrs,Cabin_B,Cabin_D,Cabin_N,PassengerId
0,0.0,52.0000,0,1,0,1,0,0,0,0,476
1,1.0,10.5000,0,0,1,0,0,0,0,1,57
2,1.0,512.3292,0,0,1,0,0,0,0,1,259
3,1.0,30.0000,0,1,0,1,0,0,1,0,741
4,0.0,110.8833,0,1,0,1,0,0,0,0,699


### Separar dados em atributos (X) e classe (y)

In [ ]:
X = dados.drop(columns=['PassengerId', 'Survived'])
y = dados['Survived']
X.head()

,Fare,Pclass_3,Sex_male,Name_Miss,Name_Mr,Name_Mrs,Cabin_B,Cabin_D,Cabin_N
0,52.0000,0,1,0,1,0,0,0,0
1,10.5000,0,0,1,0,0,0,0,1
2,512.3292,0,0,1,0,0,0,0,1
3,30.0000,0,1,0,1,0,0,1,0
4,110.8833,0,1,0,1,0,0,0,0


### Lista com todas as técnicas que serão utilizadas

In [ ]:
tecnicas = [ ('AdaBoostClassifier', sklearn.ensemble.AdaBoostClassifier()),
             ('BaggingClassifier', sklearn.ensemble.BaggingClassifier()),
             ('BernoulliNB', naive_bayes.BernoulliNB()),
             ('CalibratedClassifierCV', sklearn.calibration.CalibratedClassifierCV()),
             ('DecisionTreeClassifier', sklearn.tree.DecisionTreeClassifier()),
             ('ExtraTreeClassifier', sklearn.tree.ExtraTreeClassifier()),
             ('ExtraTreesClassifier', sklearn.ensemble.ExtraTreesClassifier()),
             #('GaussianNB', sklearn.naive_bayes.GaussianNB()),
             ('GaussianProcessClassifier', sklearn.gaussian_process.GaussianProcessClassifier()),
             ('GradientBoostingClassifier', sklearn.ensemble.GradientBoostingClassifier()),
             ('KNeighborsClassifier', sklearn.neighbors.KNeighborsClassifier()),
             ('LabelPropagation', sklearn.semi_supervised.LabelPropagation()),
             ('LabelSpreading', sklearn.semi_supervised.LabelSpreading()),
             ('LinearDiscriminantAnalysis', sklearn.discriminant_analysis.LinearDiscriminantAnalysis()),
             #('LinearSVC', sklearn.svm.LinearSVC()),
             ('LogisticRegression', sklearn.linear_model.LogisticRegression(penalty='l2')),
             ('LogisticRegressionCV', sklearn.linear_model.LogisticRegressionCV()),
             ('MLPClassifier', sklearn.neural_network.MLPClassifier()),
             ('MultinomialNB', sklearn.naive_bayes.MultinomialNB()),
             #('NearestCentroid', sklearn.neighbors.NearestCentroid()),
             #('NuSVC', sklearn.svm.classes.NuSVC()), # erro de outlier
             #('PassiveAggressiveClassifier', sklearn.linear_model.PassiveAggressiveClassifier()),
             #('Perceptron', sklearn.linear_model.Perceptron()),
             ('QuadraticDiscriminantAnalysis', sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis()),
             ('RadiusNeighborsClassifier', sklearn.neighbors.RadiusNeighborsClassifier()),
             ('RandomForestClassifier', sklearn.ensemble.RandomForestClassifier())#,
             #('RidgeClassifier', sklearn.linear_model.RidgeClassifier()),
             #('RidgeClassifierCV', sklearn.linear_model.RidgeClassifierCV()),
             #('SGDClassifier', sklearn.linear_model.SGDClassifier()),
             #('SVC', sklearn.svm.classes.SVC())
           ]

    
len(tecnicas)

20

In [ ]:
def log(mensagem):
    """
    Função para receber uma mensagem e exibir.
    Futuramente esta função pode receber a funcionalidade de salvar as mensagens de log em arquivo
    """
    print(str(datetime.now(pytz.timezone('America/Sao_Paulo'))) + ': ' + mensagem)

In [ ]:
def getNumeroAleatorio(tipo, maximo=None):
    """
    Função para retornar um número aleatório de acordo com o tipo que é passado por parâmetro.        
    """
    
    if tipo == 'cross_validation': 
        return random.randint(3,10) # Definido que a quantidade de divisão dos dados (split) será entre 3 a 10 partes (folds)
    elif tipo == 'random_state':
        return random.randint(1, 42) # Valor do random state poderá variar entre 1 e 42
    elif tipo == 'simples':  # Retorna um número entre 0 e o valor passado por parâmetro
        return random.randint(0,maximo)

In [ ]:
def splitDados(X, y):
    """
    Função para separar os dados de treino e teste, escolhendo aleatoriamente qual será a forma de separar os dados.
    Formas disponíveis de separação:
        - 80% para treino e 20% para teste
        - 75% para treino e 25% para teste
        - 70% para treino e 30% para teste
    """
    
    aleatorio = getNumeroAleatorio('simples', 2)
    
    random_state = getNumeroAleatorio('random_state')
    
    if aleatorio == 0:
        tipo_split = 'train_test_split 20%'
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
    elif aleatorio == 1:
        tipo_split = 'train_test_split 25%'
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=random_state)
    elif aleatorio == 2:
        tipo_split = 'train_test_split 30%'
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)
    
    return X_train, X_test, y_train, y_test, tipo_split, random_state

In [ ]:
def getNormalizador():
    """
    Retorna aleatoriamente qual normalizador será utilizado ou se não será utilizado normalizador.
    Se algum normalizador for escolhido, é retornado a lista de parâmetros do normalizador escolhido.
    Normalizadores disponíveis:
        - StandardScaler
        - RobustScaler
        - MinMaxScaler
        - Normalizer
    """
    
    normalizador = [(),
                    ('standard_scaler', StandardScaler()),
                    ('robust_scaler', RobustScaler()),
                    ('min_max_scaler', MinMaxScaler()),
                    ('normalizer', Normalizer())
                   ]
    
    aleatorio = getNumeroAleatorio('simples', len(normalizador)-1)
    
    if aleatorio == 0:
        parametros = {}
    elif normalizador[aleatorio][0] == 'standard_scaler':
        parametros = {
            'standard_scaler__with_mean': [True, False],
            'standard_scaler__with_std': [True, False]
        }
    elif normalizador[aleatorio][0] == 'robust_scaler':
        parametros = {
            'robust_scaler__with_centering': [True, False],
            'robust_scaler__with_scaling': [True, False]
        }
    elif normalizador[aleatorio][0] == 'min_max_scaler':
        parametros = {
            'min_max_scaler__feature_range': [(0,1), (1,10), (1,100)]
        }
    elif normalizador[aleatorio][0] == 'normalizer':
        parametros = {
            'normalizer__norm': ('l1', 'l2', 'max')
        }
        
    
    return normalizador[aleatorio], parametros

In [ ]:
def getRedutorDimensionalidade():
    """
    Retorna aleatoriamente qual redutor de dimensionalidade será utilizado ou se não será utilizado redutor de dimensionalidade.
    Se for escolhido um redutor de dimensionalidade, é retornado a lista de parâmetros do redutor de dimensionalidade escolhido.
    Redutor de dimensionalidade disponível:
        - PCA
        - NMF
        - FastICA
    """    
    
    redutor_dimensionalidade = [(),
                                ('pca', PCA()),
                                ('nmf', NMF()),
                                ('FastICA', FastICA())
                                ]
    
    aleatorio = getNumeroAleatorio('simples', len(redutor_dimensionalidade)-1)
    
    if aleatorio == 0:
        parametros = {}
    elif redutor_dimensionalidade[aleatorio][0] == 'pca':
        parametros = {
            'pca__n_components': [None, 3, 5, 7, 9, 11, 13],
            'pca__whiten': [True, False],
            'pca__svd_solver': ('auto', 'full', 'randomized')
        }
    elif redutor_dimensionalidade[aleatorio][0] == 'nmf':
        parametros = {
            'nmf__n_components': [None, 3, 5, 7, 9, 11, 13],
            'nmf__init': ('random', 'nndsvd', 'nndsvda', 'nndsvdar'),
            'nmf__solver': ('cd', 'mu')
        }
    elif redutor_dimensionalidade[aleatorio][0] == 'FastICA':
        parametros = {
            'FastICA__n_components': [None, 3, 5, 7, 9, 11, 13],
            'FastICA__algorithm': ('parallel', 'deflation'),
            'FastICA__whiten': [True, False]
        }
        
    
    return redutor_dimensionalidade[aleatorio], parametros

In [ ]:
def getParametrosTecnica(tecnica):
  """
    Recebe o nome de uma técnica de predição por parâmetro, e retorna os parâmetros da técnica
    """
  if tecnica == 'AdaBoostClassifier':
      parametros = {
          'AdaBoostClassifier__n_estimators': [5,25,50,75,100],
          'AdaBoostClassifier__learning_rate': [0.5, 1.0, 1.5],
          'AdaBoostClassifier__algorithm': ('SAMME', 'SAMME.R')
      }
  elif tecnica == 'BaggingClassifier':
      parametros = {
          'BaggingClassifier__n_estimators': [5, 10, 15],
          #'BaggingClassifier__bootstrap': [True, False],
          'BaggingClassifier__bootstrap_features': [True, False],
          'BaggingClassifier__oob_score': [True, False]
      }
  elif tecnica == 'BernoulliNB':
      parametros = {
          'BernoulliNB__alpha': [0, 0.5, 1.0, 1.5, 2],
          'BernoulliNB__fit_prior': [True, False]
      }
  elif tecnica == 'CalibratedClassifierCV':
      parametros = {
          'CalibratedClassifierCV__method': ('sigmoid', 'isotonic'),
          'CalibratedClassifierCV__cv': [3, 5, 10]
      }
  elif tecnica == 'DecisionTreeClassifier':
      parametros = {
          'DecisionTreeClassifier__criterion': ('gini', 'entropy'),
          'DecisionTreeClassifier__splitter': ('best', 'random')            
      }
  elif tecnica == 'ExtraTreeClassifier':
      parametros = {
          'ExtraTreeClassifier__criterion': ('gini', 'entropy'),
          'ExtraTreeClassifier__max_features': ('auto', 'sqrt', 'log2')
      }
  elif tecnica == 'ExtraTreesClassifier':
      parametros = {
          'ExtraTreesClassifier__criterion': ('gini', 'entropy'),
          'ExtraTreesClassifier__max_features': ('auto', 'sqrt', 'log2')
      }
  elif tecnica == 'GaussianProcessClassifier':
      parametros = {
          'GaussianProcessClassifier__n_restarts_optimizer': [0,5,10,15],
          'GaussianProcessClassifier__max_iter_predict': [10, 100, 1000]
      }  
  elif tecnica == 'GradientBoostingClassifier':
      parametros = {
          'GradientBoostingClassifier__loss': ('deviance', 'exponential'),
          'GradientBoostingClassifier__learning_rate': [0.1,0.2,0.001],
          'GradientBoostingClassifier__criterion': ('friedman_mse', 'mse', 'mae'),
          'GradientBoostingClassifier__max_features': ('auto', 'sqrt', 'log2')
      }
  elif tecnica == 'KNeighborsClassifier':
      parametros = {
          'KNeighborsClassifier__n_neighbors': [3,5,7,9],
          'KNeighborsClassifier__weights': ('uniform', 'distance'),
          'KNeighborsClassifier__algorithm': ('auto', 'ball_tree', 'kd_tree', 'brute'),
          'KNeighborsClassifier__p': [1,2]
      }
  elif tecnica == 'LabelPropagation':
      parametros = {
          'LabelPropagation__kernel': ('knn', 'rbf'),
          'LabelPropagation__n_neighbors': [3,5,7,9],
          'LabelPropagation__max_iter': [100, 1000, 10000],
          'LabelPropagation__tol': [0.01, 0.001, 0.0001]
      }
  elif tecnica == 'LabelSpreading':
      parametros = {
          'LabelSpreading__kernel': ('knn', 'rbf'),
          'LabelSpreading__n_neighbors': [3,5,7,9],
          'LabelSpreading__max_iter': [100, 1000, 10000],
          'LabelSpreading__tol': [0.01, 0.001, 0.0001]
      }
  elif tecnica == 'LinearDiscriminantAnalysis':
      parametros = {
          'LinearDiscriminantAnalysis__solver': ('svd', 'lsqr'),
          #'LinearDiscriminantAnalysis__shrinkage': (None, 'auto'),
          'LinearDiscriminantAnalysis__tol': [0.001, 0.0001, 0.00001]
      }
  elif tecnica == 'LinearSVC':
      parametros = {
          #'LinearSVC__penalty': ('l1', 'l2'),
          'LinearSVC__loss': ('hinge', 'squared_hinge'),
          #'LinearSVC__dual': [True, False],
          'LinearSVC__tol': [0.001, 0.0001, 0.00001],
          'LinearSVC__multi_class': ('ovr', 'crammer_singer'),
          'LinearSVC__fit_intercept': [True, False],
          'LinearSVC__max_iter': [100, 1000, 10000]
      }
  elif tecnica == 'LogisticRegression':
      parametros = {
          #'LogisticRegression__penalty': ('l2'),
          'LogisticRegression__fit_intercept': [True, False],
          'LogisticRegression__solver': ('newton-cg', 'sag', 'lbfgs'),
          'LogisticRegression__max_iter': [10, 100, 1000, 10000],
          'LogisticRegression__warm_start': [True, False]
      }
  elif tecnica == 'LogisticRegressionCV':
      parametros = {
          'LogisticRegressionCV__fit_intercept': [True, False],
          'LogisticRegressionCV__cv': ['warn', 3, 5, 7, 9],
          'LogisticRegressionCV__solver': ('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'),
          'LogisticRegressionCV__refit': [True, False],
          'LogisticRegressionCV__multi_class': ('ovr', 'auto')
      }
  elif tecnica == 'MLPClassifier':
      parametros = {
          'MLPClassifier__hidden_layer_sizes': [(1,), (100,), (500,), (1,3), (100,3), (500,3)],
          'MLPClassifier__activation': ('identity', 'logistic', 'tanh', 'relu'),
          'MLPClassifier__solver': ('lbfgs', 'sgd', 'adam'),
          'MLPClassifier__learning_rate': ('constant', 'invscaling', 'adaptive'),
          'MLPClassifier__shuffle': [True, False],
          'MLPClassifier__max_iter': [100, 500, 1000],
          'MLPClassifier__tol': [0.001, 0.0001, 0.00001]
      }
  elif tecnica == 'MultinomialNB':
      parametros = {
          'MultinomialNB__alpha': [0.0, 1.0],
          'MultinomialNB__fit_prior': [True, False]
      }
  elif tecnica == 'NuSVC':
      parametros = {
          'NuSVC__kernel': ('linear', 'poly', 'rbf', 'sigmoid', 'precomputed'),
          'NuSVC__gamma': ('rbf', 'poly', 'sigmoid'),
          'NuSVC__shrinking': [True, False],
          'NuSVC__decision_function_shape': ('ovo', 'ovr')
      }
  elif tecnica == 'PassiveAggressiveClassifier':
      parametros = {
          'PassiveAggressiveClassifier__fit_intercept': [True, False],
          'PassiveAggressiveClassifier__shuffle': [True, False],
          'PassiveAggressiveClassifier__loss': ('hinge', 'squared_hinge'),
          'PassiveAggressiveClassifier__average': [True, False]
      }
  elif tecnica == 'Perceptron':
      parametros = {
          'Perceptron__penalty': (None, 'l1', 'l2', 'elasticnet'),
          'Perceptron__shuffle': [True, False]
      }
  elif tecnica == 'QuadraticDiscriminantAnalysis':
      parametros = {
          'QuadraticDiscriminantAnalysis__store_covariance': [True, False],
          'QuadraticDiscriminantAnalysis__store_covariances': [True, False],
          'QuadraticDiscriminantAnalysis__tol': [0.001, 0.0001, 0.00001]
      }
  elif tecnica == 'RadiusNeighborsClassifier':
      parametros = {
          'RadiusNeighborsClassifier__weights': ('uniform', 'distance'),
          'RadiusNeighborsClassifier__algorithm': ('ball_tree', 'kd_tree', 'brute', 'auto'),
          'RadiusNeighborsClassifier__p': [1, 2]
      }
  elif tecnica == 'RandomForestClassifier':
      parametros = {
          'RandomForestClassifier__criterion': ('gini', 'entropy'),
          'RandomForestClassifier__max_features': ('auto', 'sqrt', 'log2', None),
          #'RandomForestClassifier__bootstrap': [True, False],
          #'RandomForestClassifier__oob_score': [True, False],
          'RandomForestClassifier__warm_start': [True, False],
          'RandomForestClassifier__class_weight': ('balanced', 'balanced_subsample', None)
      }
  elif tecnica == 'RidgeClassifier':
      parametros = {
          'RidgeClassifier__fit_intercept': [True, False],
          'RidgeClassifier__normalize': [True, False],
          'RidgeClassifier__solver': ('auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'),
          'RidgeClassifier__tol': [0.01, 0.001, 0.0001]
      }
  elif tecnica == 'RidgeClassifierCV':
      parametros = {
          'RidgeClassifierCV__fit_intercept': [True, False],
          'RidgeClassifierCV__normalize': [True, False],
          'RidgeClassifierCV__cv': [3,5,7,9]
      }
  elif tecnica == 'SGDClassifier':
      parametros = {
          'SGDClassifier__loss': ('hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'),
          'SGDClassifier__shuffle': [True, False],
          #'SGDClassifier__learning_rate': ['constant', 'optimal', 'invscaling'],
          'SGDClassifier__average': [True, False]
      }
  
      
  return parametros

In [ ]:
def salvarDesempenho(resultados):
    """
    Função que recebe um Pandas DataFrame por parâmetro, e salva os dados no arquivo CSV.
    Caso já exista um arquivo, os dados recebidos por parâmetro são adicionados no arquivo
    """
    
    nome_arquivo = 'resultados_treino.csv'    
    
    try:
        csv_resultados = pd.read_csv(nome_arquivo, sep=';') # Abrir arquivo existente
        csv_resultados = pd.concat([csv_resultados, resultados]) # Adicionar dados recebidos por parâmetro ao conjunto de dados já existente
        csv_resultados.to_csv(nome_arquivo, sep=';', index=False) # Salvar os dados 
    except FileNotFoundError: # Caso não exista o arquivo, será criado um novo com os dados recebido por parâmetro
        resultados.to_csv(nome_arquivo, sep=';', index=False)

In [ ]:
def avaliarDesempenho(tecnica, y_true, y_pred, y_proba):
    """
    Função que recebe o valor verdadeiro/real (y_true) e o valor predito (y_pred), 
    aplica os avaliadores de desempenho e retorna o resultado dos avaliadores de desempenho.
    Avaliadores de desempenho aplicados:
        - accuracy
        - roc_auc_score
        - auc
        - f1_score
        - log_loss
        - precision
        - recall
    """
    
    resultados = {}
    resultados['tecnica'] = [tecnica]
    # Aplicando avaliadores de desempenho        
    resultados['accuracy'] = [sklearn.metrics.accuracy_score(y_true, y_pred)]
    resultados['roc_auc_score'] = [sklearn.metrics.roc_auc_score(y_true, y_pred)]
    fp, tp, thresholds = sklearn.metrics.roc_curve(y_true, y_proba[:, 1])
    resultados['auc'] = [sklearn.metrics.auc(fp, tp)]
    resultados['f1_score'] = [sklearn.metrics.f1_score(y_true, y_pred)]
    resultados['log_loss'] = [sklearn.metrics.log_loss(y_true, y_pred)]
    resultados['precision'] = [sklearn.metrics.precision_score(y_true, y_pred)]
    resultados['recall'] = [sklearn.metrics.recall_score(y_true, y_pred)]
        
    log('Técnica: {0} - accuracy: {1}, roc_auc_score: {2}, auc: {3}'.format(
            tecnica, resultados['accuracy'], resultados['roc_auc_score'], resultados['auc']) )
    
    pd_resultados = pd.DataFrame(data=resultados)   
    
    return pd_resultados    

In [ ]:
def autoModeling(X, y, quantidade_execucoes):
    """
    Função que executa todas as etapas necessárias para separar os dados em treino e teste, e montar o pipeline de treinamento
    O pipeline é composto de:
        - Normalizador e seus parâmetros, se existente
        - Redutor de dimensionalidade e seus parâmetros, se existente
        - Técnica de predição e seus parâmetros
    Após o pipeline estar montado, é realizado uma busca aleatória (Random Search) para encontrar a melhor combinação de parâmetros, essa busca aleatória é executada por 4 vezes
    Após feito a busca aleatória pela melhor combinação de parâmetros, é realizado a predição do 'X_test', para posteriormente aplicar a avaliação de desempenho do modelo de predição
    Aplicado a avaliação de desempenho, seus resultados são salvos em um arquivo .csv
    Posteriomente, o modelo gerado é salvo (.pkl)
    """

    for i in range(1,quantidade_execucoes+1):
    
        log('========== Iniciando execução número: ' + str(i) + ' de ' + str(quantidade_execucoes))

        X_train, X_test, y_train, y_test, tipo_split, random_state_split = splitDados(X, y) # Separando os dados em treino e teste


        pipeline = []
        parametros = {}

        normalizador_pipeline, normalizador_parametros = getNormalizador() # Coletando o normalizador, que será definido aleatoriamente
        normalizador_nome = 'Não utilizado'
        if len(normalizador_pipeline) > 0: # Se for definido que terá normalizador, o normalizador é adicionado no pipeline e no array que salva os parâmetros
            pipeline += [normalizador_pipeline]
            parametros.update(normalizador_parametros)
            normalizador_nome = normalizador_pipeline[0]

        redutor_dimensionalidade_pipeline, redutor_dimensionalidade_parametros = getRedutorDimensionalidade() # Coletando o redutor de dimensionalidade, que será definido aleatoriamente
        redutor_dimensionalidade_nome = 'Não utilizado'
        if len(redutor_dimensionalidade_pipeline) > 0: # Se for definido que terá redutor de dimensionalidade, o redutor de dimensionalidade é adicionado no pipeline e no array que salva os parâmetros
            pipeline += [redutor_dimensionalidade_pipeline]
            parametros.update(redutor_dimensionalidade_parametros)
            redutor_dimensionalidade_nome = redutor_dimensionalidade_pipeline[0]


        log('========== Iniciando treinamento com normalizador: {0}, redutor_dimensionalidade: {1}'.format(normalizador_nome, redutor_dimensionalidade_nome))
        

        for tecnica, model in tecnicas: # Para cada técnica presente na variável "tecnicas_regressao", coletar os parametros da técnica, realizar o random search e avaliar o seu desempenho
            log('Técnica: ' + tecnica)
            try:

                pipeline_tecnica = pipeline.copy()
                pipeline_tecnica += [(tecnica, model)] # Adicionando a técnica no pipeline

                parametros_tecnica = parametros.copy()
                parametros_tecnica.update(getParametrosTecnica(tecnica)) # Coletando os parâmetros da técnica e adicionando no array de parâmetros

                pipeline_final = Pipeline(pipeline_tecnica)

                cross_validation = getNumeroAleatorio('cross_validation') # Coletando um número aleatório para representar o cross_validation do Random Search
                random_state = getNumeroAleatorio('random_state') # Coletando um número aleatório para representar o random_state do Random Search

                # Executando o Random Search com 4 iterações
                modelo = RandomizedSearchCV(n_iter=10, scoring='roc_auc', estimator=pipeline_final, param_distributions=parametros_tecnica, cv=cross_validation, random_state=random_state, n_jobs= -1)
                #modelo = RandomizedSearchCV(n_iter=4, estimator=pipeline_final, param_distributions={}, cv=cross_validation, random_state=random_state, n_jobs= -1)

                modelo.fit(X_train, y_train)

                y_pred = modelo.predict(X_test) # Realizando a predição para os dados de teste
                y_proba = modelo.predict_proba(X_test) # Retornando a probabilidade da predição para os dados de teste
                
                desempenho_tecnica = avaliarDesempenho(tecnica=tecnica, y_true=y_test, y_pred=y_pred, y_proba=y_proba) # Chamando a função que avalia o desempenho da técnica

                # Adicionando informações relevantes e informações para identificar como é o modelo gerado, permitindo que seja reproduzido manualmente caso necessário
                identificador_aleatorio1 = getNumeroAleatorio('simples', 9999999) # Gerar um identificador aleatório para o resultado dessa técnica
                identificador_aleatorio2 = getNumeroAleatorio('simples', 9999999)
                desempenho_tecnica['identificador_aleatorio1'] = [identificador_aleatorio1]
                desempenho_tecnica['identificador_aleatorio2'] = [identificador_aleatorio2]
                desempenho_tecnica['tipo_split'] = [tipo_split]
                desempenho_tecnica['normalizador'] = [normalizador_nome]
                desempenho_tecnica['redutor_dimensionalidade'] = [redutor_dimensionalidade_nome]
                desempenho_tecnica['pipeline'] = [str(pipeline_tecnica)]
                desempenho_tecnica['parametros'] = [str(parametros_tecnica)]
                desempenho_tecnica['random_state_split'] = [random_state_split]
                desempenho_tecnica['random_state_random_search'] = [random_state]
                desempenho_tecnica['cross_validation_random_search'] = [cross_validation]
                desempenho_tecnica['best_estimator'] = [modelo.best_estimator_]
                desempenho_tecnica['best_params'] = [modelo.best_params_]
                desempenho_tecnica['versao_dados'] = [versao_dados]

                salvarDesempenho(desempenho_tecnica) # Salvando o desempenho da técnica



                nome_arquivo = 'modelos/' + str(identificador_aleatorio1) + '_' + str(identificador_aleatorio2) + '_' + tecnica + '.pkl' # Criando o nome do arquivo, ex.: 112302_KNeighborsClassifier.pkl
                joblib.dump(modelo.best_estimator_, nome_arquivo) # Salvando o modelo

            except Exception as e:

                log('Erro durante execução da técnica: ' + tecnica)
                #print(e)
                pass # Passar para a próxima técnica



            del(pipeline_tecnica) # Liberar memória
            del(parametros_tecnica) # Liberar memória

        log('========== Finalizado execução número: ' + str(i) + ' de ' + str(quantidade_execucoes))


In [ ]:
# Executando por X vezes o treinamento das técnicas com os dados de treino (X e y) 
autoModeling(X, y, quantidade_execucoes=50)

2020-06-06 10:08:19.214106-03:00: ========== Iniciando execução número: 1 de 50
2020-06-06 10:08:19.217384-03:00: ========== Iniciando treinamento com normalizador: Não utilizado, redutor_dimensionalidade: Não utilizado
2020-06-06 10:08:19.217501-03:00: Técnica: AdaBoostClassifier
2020-06-06 10:08:23.338075-03:00: Técnica: AdaBoostClassifier - accuracy: [0.824], roc_auc_score: [0.8179112554112553], auc: [0.872564935064935]
2020-06-06 10:08:28.873379-03:00: Técnica: BaggingClassifier
2020-06-06 10:08:29.557727-03:00: Técnica: BaggingClassifier - accuracy: [0.848], roc_auc_score: [0.8216991341991343], auc: [0.9099025974025974]
2020-06-06 10:08:29.602392-03:00: Técnica: BernoulliNB
2020-06-06 10:08:30.141620-03:00: Técnica: BernoulliNB - accuracy: [0.816], roc_auc_score: [0.8114177489177489], auc: [0.8772997835497836]
2020-06-06 10:08:30.191230-03:00: Técnica: CalibratedClassifierCV
2020-06-06 10:08:31.778254-03:00: Técnica: CalibratedClassifierCV - accuracy: [0.808], roc_auc_score: [0.80